In [3]:
import os
from typing import Union, Tuple
from IPython.display import Audio, display, HTML
from pytube import YouTube

In [4]:
def construct_embed_url(vid_key: str, t_start: Union[int, None], t_stop: Union[int, None]) -> str:
    if t_start and t_stop:
        assert t_start < t_stop, f"Wait, the start timestamp `t_start` ({t_start}) is supposed to be less than the end timestamp `t_stop` ({t_stop})"

    if vid_key:
        embed_url = f"https://www.youtube.com/embed/{vid_key}?rel=0"
    if t_start:
        embed_url += f"&start={t_start}"
    if t_stop:
        embed_url += f"&end={t_stop}"
    embed_url += "&;controls=0&;showinfo=0"
    return embed_url

# Create a YouTube object and get the video stream with the highest resolution
def download_youtube_video_by_key(vid_key: str, 
                                  output_dir: str='', 
                                  out_filename: str='clip', 
                                  file_ext: str='mp4', 
                                  audio_only: bool=False
                                 ) -> Tuple[str, str]:
    vid_url = f'https://youtu.be/{vid_key}'

    yt = YouTube(vid_url)
    download_filename = f'{out_filename}.{file_ext}'
    
    if audio_only:
        ## get audio only
        stream = yt.streams.get_audio_only(subtype=file_ext)
    else:
        ## get both video and audio
        stream = yt.streams.get_highest_resolution()

    # Download the video
    stream.download(output_dir, filename=download_filename)
    
    return output_dir, download_filename

# Use ffmpeg to extract the desired section of the video and audio
def crop_clip(output_dir: str, input_filename: str, output_filename: str=None) -> Tuple[str, str]:
    if output_filename is None:
        filename, ext = input_filename.split('.')
        output_filename = f'{filename}_cropped.{ext}'
    cmd = f"yes y | ffmpeg -i {os.path.join(output_dir, input_filename)} -ss {t_start} -to {t_stop} -c copy {os.path.join(output_dir, output_filename)}"
    os.system(cmd)
    return output_dir, output_filename

In [10]:
vid_key = input("YouTube video hash (e.g. 'ZmUENUZx2w0' for 'https://youtu.be/ZmUENUZx2w0'): ")
assert vid_key is not None and isinstance(vid_key, str) and len(vid_key) > 0, "We need the video hash here, please."

YouTube video hash (e.g. 'ZmUENUZx2w0' for 'https://youtu.be/ZmUENUZx2w0'):  5JAAZZG2V4k


In [11]:
t_start = input("time starts (second), please leave it blank for None: ") or None
t_stop = input("time ends (second), please leave it blank for None: ") or None

t_start = int(t_start) if t_start else None
t_stop = int(t_stop) if t_stop else None

print(f'video start:{t_start} and end:{t_stop}')

time starts (second), please leave it blank for None:  
time ends (second), please leave it blank for None:  


video start:None and end:None


In [12]:
output_dir = input("output directory e.g. '~/Downloads/Love_On_Top': ") or 'temp_out'
output_dir = os.path.expanduser(output_dir)
print(f'output_dir: {output_dir}')

output directory e.g. '~/Downloads/Love_On_Top':  ~/Downloads/dhamma/5JAAZZG2V4k


output_dir: /Users/kittipat.kampa/Downloads/dhamma/5JAAZZG2V4k


In [13]:
embed_url = construct_embed_url(vid_key, t_start, t_stop)
print(embed_url)

https://www.youtube.com/embed/5JAAZZG2V4k?rel=0&;controls=0&;showinfo=0


In [14]:
# Display YouTube
print('Preview')
HTML('<iframe width="560" height="315" src=' + embed_url + 'frameborder="0" allowfullscreen></iframe>')

Preview


In [15]:
output_dir, download_filename = download_youtube_video_by_key(vid_key, output_dir=output_dir)
# Show a success message
print(f"The video has been downloaded to {os.path.join(output_dir, download_filename)}.")

The video has been downloaded to /Users/kittipat.kampa/Downloads/dhamma/5JAAZZG2V4k/clip.mp4.


In [16]:
# crop the clip
if (t_start is not None) or (t_stop is not None):
    output_dir, output_filename = crop_clip(output_dir, download_filename)
else:
    output_filename = download_filename

clip_path = os.path.join(output_dir, output_filename)
print(f'clip_path: {clip_path}')

clip_path: /Users/kittipat.kampa/Downloads/dhamma/5JAAZZG2V4k/clip.mp4


## Extract mp3 from downloaded video file

In [17]:
from moviepy.video.io.VideoFileClip import VideoFileClip

In [18]:
def change_file_extension(input_path: str, new_ext: str):
    # construct output mp3 path
    path_tokens = input_path.split('.')
    path_tokens[-1] = new_ext
    output_path = '.'.join(path_tokens)
    return output_path
    
def extract_mp3_from_video(vid_input_path: str):
    mp3_output_path = change_file_extension(vid_input_path, 'mp3')
    print(f"input_path: {vid_input_path} -> output_path: {mp3_output_path}")

    # Extract the audio from the video
    video = VideoFileClip(vid_input_path)
    audio = video.audio

    # Save the audio to a file
    print('converting to mp3...')
    audio.write_audiofile(mp3_output_path)
    print(f'Writing to mp3 file {mp3_output_path} completed')
    
    return mp3_output_path

In [19]:
vid_input_path = clip_path
mp3_output_path = extract_mp3_from_video(vid_input_path)

input_path: /Users/kittipat.kampa/Downloads/dhamma/5JAAZZG2V4k/clip.mp4 -> output_path: /Users/kittipat.kampa/Downloads/dhamma/5JAAZZG2V4k/clip.mp3
converting to mp3...
MoviePy - Writing audio in /Users/kittipat.kampa/Downloads/dhamma/5JAAZZG2V4k/clip.mp3


MoviePy - Done.
Writing to mp3 file /Users/kittipat.kampa/Downloads/dhamma/5JAAZZG2V4k/clip.mp3 completed


## Transcibe mp3 file

### openai

In [20]:
import openai
import streamlit as st
openai.api_key = st.secrets["OPENAI_API_KEY"]

In [21]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
audio_file= open("/Users/kittipat.kampa/Downloads/dhamma/5JAAZZG2V4k/clip.mp3", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)

In [23]:
transcript.to_dict()

{'text': 'ความมือกลอมเกิดไป ความมือกลอมเกิดไป สังคมมันเปลี่ยนเร็วนะช่วงนี้ คนโบราณ รุ่น ปู่ ยาตายายลงพ่อน เกิดมายังไง ตายไปแบบนั้น บ้านเมืองไม่ค่อยเปลี่ยนแปลง ก็เปลี่ยนไปเรื่อยๆ มาถึงวันนี้ก็เปลี่ยนครั้งใหญ่นะ มีโควิดเข้ามา การดำลงชีวิตก็เปลี่ยนหมด อาชีพการงานบางอย่างก็หายไป บางอย่างก็เกิดขึ้นใหม่ คนไหนปรับตัวได้ ตามทันได้ ก็อยู่รอด ปรับตัวไม่ได้ ตามไม่ทัน ไม่เข้าใจ ก็อยู่ไม่ได้ เป็นหลับ ของการ อะไร พัฒนาการ เหมือนสัตว์ทั้งหลาย ดินฟ้า อากาศเปลี่ยนไป ปรับตัวได้ มันก็อยู่รอดมา ปรับตัวไม่ได้ หุบผ่าวอันนั้นก็หมดไป นี่เราไปควบคุม มังครับ ไม่ให้โลกมันเปลี่ยนแรง เราทำไม่ได้ เหตุปัจจั้ยต่าง ๆ มากมายที่ทำให้มันเปลี่ยน ยิ่งกํลงของกิแลจแรงเท่าไหร่ มันก็ยิ่งดินโรนที่จะให้เปลี่ยนแรง เร็วเท่านั้น นี่ถ้าเราหยับ้ายังมันไม่ได้ เราทำความเข้าใจมัน โลกนี้ต้องเปลี่ยนแรง ทุกอย่างต้องเปลี่ยนแรง สิ่งทั้งหลาย รวมแต่ไม่ที่อย่าง ฝีมวยว่าธัมมาเท่านั้นที่มันไม่เปลี่ยนแรง แล้วโลกมันเปลี่ยนแรง แต่ธัมมามันไม่เปลี่ยนแรง อย่างไรมันก็ไม่เทียง อย่างไรมันก็ทนอยู่ไม่ได้ ศพหวักท้างหลาย อย่างวันเทียวที่แล้วนิรลณภาพมาหากินนะ 

### GCP

We use speech-to-text

In [2]:
# Import the Speech-to-Text client library
from google.cloud import speech
from google.protobuf import wrappers_pb2

# Instantiates a client
client=speech.SpeechClient()

# The name of the audio file to transcribe
gcs_uri="gs://raml-sandbox-prod/audio-files/clip.mp3"

def transcribe_speech():
    audio=speech.RecognitionAudio(uri=gcs_uri)
    
    config=speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.ENCODING_UNSPECIFIED,
    sample_rate_hertz=8000,
    language_code="th-TH",
    model="default",
    enable_word_time_offsets=True,
    max_alternatives=3,
    )
    
    # Detects speech in the audio file
    operation=client.long_running_recognize(config=config, audio=audio)
    
    print("Waiting for operation to complete...")
    response=operation.result(timeout=90)
    
    for result in response.results:
        print("Transcript: {}".format(result.alternatives[0].transcript))

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
transcribe_speech()

### AWS

We use AWS transcribe